In [4]:

# Course: ACE6233 – Data Preprocessing & Model Evaluation
# Scope: DATA PREPROCESSING
# Dataset: Airline Delay Dataset

In [5]:

# ------------------------------------------------------
# 1. IMPORT REQUIRED LIBRARIES
# ------------------------------------------------------
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression

In [6]:
# Load dataset
df = pd.read_csv("airline_delay.csv")

In [7]:
# Display first 5 rows
df.head()

,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,weather_ct,...,security_ct,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2023,8,9E,Endeavor Air Inc.,ABE,"Allentown/Bethlehem/Easton, PA: Lehigh Valley ...",89.0,13.0,2.25,1.60,...,0.0,5.99,2.0,1.0,1375.0,71.0,761.0,118.0,0.0,425.0
1,2023,8,9E,Endeavor Air Inc.,ABY,"Albany, GA: Southwest Georgia Regional",62.0,10.0,1.97,0.04,...,0.0,7.42,0.0,1.0,799.0,218.0,1.0,62.0,0.0,518.0
2,2023,8,9E,Endeavor Air Inc.,AEX,"Alexandria, LA: Alexandria International",62.0,10.0,2.73,1.18,...,0.0,4.28,1.0,0.0,766.0,56.0,188.0,78.0,0.0,444.0
3,2023,8,9E,Endeavor Air Inc.,AGS,"Augusta, GA: Augusta Regional at Bush Field",66.0,12.0,3.69,2.27,...,0.0,1.57,1.0,1.0,1397.0,471.0,320.0,388.0,0.0,218.0
4,2023,8,9E,Endeavor Air Inc.,ALB,"Albany, NY: Albany International",92.0,22.0,7.76,0.00,...,0.0,11.28,2.0,0.0,1530.0,628.0,0.0,134.0,0.0,768.0


In [8]:
# 3. DATASET OVERVIEW
# Shape of dataset
print("Dataset shape:", df.shape)

Dataset shape: (171666, 21)


In [9]:
# Data types and non-null counts
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171666 entries, 0 to 171665
Data columns (total 21 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   year                 171666 non-null  int64  
 1   month                171666 non-null  int64  
 2   carrier              171666 non-null  object 
 3   carrier_name         171666 non-null  object 
 4   airport              171666 non-null  object 
 5   airport_name         171666 non-null  object 
 6   arr_flights          171426 non-null  float64
 7   arr_del15            171223 non-null  float64
 8   carrier_ct           171426 non-null  float64
 9   weather_ct           171426 non-null  float64
 10  nas_ct               171426 non-null  float64
 11  security_ct          171426 non-null  float64
 12  late_aircraft_ct     171426 non-null  float64
 13  arr_cancelled        171426 non-null  float64
 14  arr_diverted         171426 non-null  float64
 15  arr_delay        

In [10]:
# Statistical summary for numerical columns
df.describe()

,year,month,arr_flights,arr_del15,carrier_ct,weather_ct,nas_ct,security_ct,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
count,171666.000000,171666.000000,171426.000000,171223.000000,171426.000000,171426.000000,171426.000000,171426.000000,171426.000000,171426.000000,171426.000000,171426.00000,171426.000000,171426.000000,171426.000000,171426.000000,171426.000000
mean,2018.551361,6.493633,362.528467,66.434387,20.796615,2.250347,19.381147,0.157096,23.770554,7.530497,0.863387,4239.48733,1437.185124,222.563497,920.651704,7.382725,1651.700063
std,2.890006,3.440908,992.894662,179.540694,50.315176,7.314252,61.675244,0.717405,72.393477,43.654880,3.772853,12618.56605,4215.677812,821.086511,3423.509335,41.779985,5221.878385
min,2013.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2016.000000,4.000000,50.000000,6.000000,2.160000,0.000000,1.000000,0.000000,1.230000,0.000000,0.000000,335.00000,110.000000,0.000000,34.000000,0.000000,65.000000
50%,2019.000000,7.000000,100.000000,17.000000,6.400000,0.400000,3.910000,0.000000,5.000000,1.000000,0.000000,1018.00000,375.000000,18.000000,146.000000,0.000000,320.000000
75%,2021.000000,9.000000,250.000000,47.000000,17.260000,1.860000,11.710000,0.000000,15.260000,4.000000,1.000000,2884.00000,1109.000000,146.000000,477.000000,0.000000,1070.000000
max,2023.000000,12.000000,21977.000000,4176.000000,1293.910000,266.420000,1884.420000,58.690000,2069.070000,4951.000000,197.000000,438783.00000,196944.000000,31960.000000,112018.000000,3760.000000,227959.000000


In [11]:
# 4. CHECK & REMOVE DUPLICATES
duplicates = df.duplicated().sum()
print(f"Duplicate rows: {duplicates}")

df = df.drop_duplicates()

Duplicate rows: 0


In [12]:
# 5. IDENTIFY NUMERICAL & CATEGORICAL COLUMNS
num_cols = df.select_dtypes(include=["int64", "float64"]).columns.tolist()
cat_cols = df.select_dtypes(include=["object"]).columns.tolist()

print("Numerical Columns:", num_cols)
print("Categorical Columns:", cat_cols)

Numerical Columns: ['year', 'month', 'arr_flights', 'arr_del15', 'carrier_ct', 'weather_ct', 'nas_ct', 'security_ct', 'late_aircraft_ct', 'arr_cancelled', 'arr_diverted', 'arr_delay', 'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay']
Categorical Columns: ['carrier', 'carrier_name', 'airport', 'airport_name']


In [13]:
# Check missing values
print(df.isnull().sum())

year                     0
month                    0
carrier                  0
carrier_name             0
airport                  0
airport_name             0
arr_flights            240
arr_del15              443
carrier_ct             240
weather_ct             240
nas_ct                 240
security_ct            240
late_aircraft_ct       240
arr_cancelled          240
arr_diverted           240
arr_delay              240
carrier_delay          240
weather_delay          240
nas_delay              240
security_delay         240
late_aircraft_delay    240
dtype: int64


In [14]:
# Median imputation for numerical columns (robust to outliers)
imputer = SimpleImputer(strategy="median")
df[num_cols] = imputer.fit_transform(df[num_cols])

In [15]:
# Verify missing values handled
print(df.isnull().sum())

year                   0
month                  0
carrier                0
carrier_name           0
airport                0
airport_name           0
arr_flights            0
arr_del15              0
carrier_ct             0
weather_ct             0
nas_ct                 0
security_ct            0
late_aircraft_ct       0
arr_cancelled          0
arr_diverted           0
arr_delay              0
carrier_delay          0
weather_delay          0
nas_delay              0
security_delay         0
late_aircraft_delay    0
dtype: int64


In [16]:
# 7.DEFINE TARGET VARIABLE (REGRESSION)
y = df["arr_delay"]

# 7a. OUTLIER TREATMENT (Optional but Recommended for Regression)
# Capping at the 99th percentile to remove extreme anomalies
upper_limit = df["arr_delay"].quantile(0.99)
df["arr_delay"] = df["arr_delay"].clip(upper=upper_limit)

# Alternatively, apply log transformation:
# df["arr_delay"] = np.log1p(df["arr_delay"])

In [17]:
# NEW STEP: TARGET VARIABLE OUTLIER CAPPING
# Identify the 99th percentile to handle extreme anomalies
upper_limit = df['arr_delay'].quantile(0.99)

# Cap (clip) values at this limit to stabilize the regression model
df['arr_delay'] = df['arr_delay'].clip(upper=upper_limit)

print(f"Outlier Capping Applied: Values above {upper_limit:.2f} minutes capped.")

Outlier Capping Applied: Values above 60324.10 minutes capped.


In [18]:
# 8. REMOVE DATA LEAKAGE & REDUNDANT COLUMNS
leakage_cols = [
    "arr_del15", "carrier_delay", "weather_delay", 
    "nas_delay", "security_delay", "late_aircraft_delay"
]

# Drop names to prevent high-cardinality issues during encoding
redundant_cols = ["carrier_name", "airport_name"]

X = df.drop(columns=leakage_cols + redundant_cols + ["arr_delay"])

In [19]:
# 9. ENCODE CATEGORICAL VARIABLES
# Explicitly list the columns to encode
X = pd.get_dummies(X, columns=['carrier', 'airport'], drop_first=True)

In [20]:
# 10. TRAIN–TEST SPLIT 
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

In [21]:
# 11. FEATURE SCALING (NUMERICAL ONLY)
scaler = StandardScaler()

num_cols = X_train.select_dtypes(include=["int64", "float64"]).columns

X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
X_test[num_cols] = scaler.transform(X_test[num_cols])

In [22]:
# Align train & test columns

X_train, X_test = X_train.align(X_test, join="left", axis=1, fill_value=0)

In [23]:
#12. FEATURE SELECTION (REGRESSION)
selector = SelectKBest(score_func=f_regression, k=10)

X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)

In [24]:
#13. FINAL OUTPUT SUMMARY
print("Regression Preprocessing Completed Successfully")
print("Training feature shape:", X_train_selected.shape)
print("Testing feature shape:", X_test_selected.shape)

Regression Preprocessing Completed Successfully
Training feature shape: (137332, 10)
Testing feature shape: (34334, 10)


In [25]:
# Scope: MODEL TRAINING

In [26]:
# ---------------------------------------------------------
# REGRESSION MODEL SELECTION
# ---------------------------------------------------------


from sklearn.linear_model import LinearRegression, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor


In [27]:
# ---------------------------------------------------------
# MODEL TRAINING AND BASELINE EVALUATION
# ---------------------------------------------------------


from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


models = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(alpha=1.0),
    "Decision Tree Regressor": DecisionTreeRegressor(
        max_depth=10,
        random_state=42
    ),
    "Random Forest Regressor": RandomForestRegressor(
        n_estimators=100,
        max_depth=15,
        random_state=42,
        n_jobs=-1
    ),
    "Gradient Boosting Regressor": GradientBoostingRegressor(
        n_estimators=100,
        learning_rate=0.1,
        random_state=42
    )
}

results = []

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    results.append({
        "Model": name,
        "MAE": mean_absolute_error(y_test, y_pred),
        "RMSE": np.sqrt(mean_squared_error(y_test, y_pred)),
        "R2 Score": r2_score(y_test, y_pred)
    })

results_df = pd.DataFrame(results)
results_df.sort_values("RMSE")


,Model,MAE,RMSE,R2 Score
3,Random Forest Regressor,590.997981,2154.018782,0.971472
4,Gradient Boosting Regressor,681.199357,2197.843202,0.970300
0,Linear Regression,1004.705379,2820.794339,0.951077
1,Ridge Regression,1004.422829,2820.801013,0.951077
2,Decision Tree Regressor,800.066178,3096.946239,0.941029


In [28]:
# ---------------------------------------------------------
# HYPERPARAMETER TUNING (ULTRA-FAST VERSION)
# ---------------------------------------------------------


from sklearn.model_selection import RandomizedSearchCV

param_dist = {
    "n_estimators": [50],
    "max_depth": [10, None]
}

rf = RandomForestRegressor(
    random_state=42,
    n_jobs=-1
)

random_search = RandomizedSearchCV(
    rf,
    param_distributions=param_dist,
    n_iter=1,      
    cv=2,          
    scoring="neg_root_mean_squared_error",
    random_state=42,
    n_jobs=-1
)

random_search.fit(X_train, y_train)

best_rf = random_search.best_estimator_
print("Best Parameters:", random_search.best_params_)


Best Parameters: {'n_estimators': 50, 'max_depth': None}


In [29]:
# ---------------------------------------------------------
# FINAL EVALUATION OF TUNED MODEL
# ---------------------------------------------------------


y_pred_best = best_rf.predict(X_test)

print("Final MAE:", mean_absolute_error(y_test, y_pred_best))
print("Final RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_best)))
print("Final R2 Score:", r2_score(y_test, y_pred_best))


Final MAE: 577.5155327081028
Final RMSE: 2134.72043506574
Final R2 Score: 0.9719811364533862


In [30]:
from sklearn.model_selection import KFold, cross_val_score
import numpy as np

kf = KFold(
    n_splits=5,
    shuffle=True,
    random_state=42
)

cv_scores = cross_val_score(
    models["Random Forest Regressor"],
    X,
    y,
    cv=kf,
    scoring="neg_root_mean_squared_error",
    n_jobs=-1
)

cv_rmse = -cv_scores

print("Cross-validation RMSE for each fold:", cv_rmse)
print("Mean CV RMSE:", cv_rmse.mean())
print("Standard Deviation:", cv_rmse.std())


Cross-validation RMSE for each fold: [2115.40251884 1763.34520898 2108.32516715 2122.48585605 1914.14272275]
Mean CV RMSE: 2004.7402947531832
Standard Deviation: 143.749419701409
